In [3]:
import spacy
import numpy as np
import pandas as pd
from nltk import word_tokenize

nlp = spacy.load('ru_core_news_sm')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [4]:
bank_data = pd.read_csv('banks.csv',sep ="\t")
bank_data.head()



X_train, X_test, y_train, y_test = train_test_split(bank_data['Text'], bank_data['Score'],test_size = 0.5, random_state=35)

def tokenize_text(text):
    doc = nlp(text) 
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return(tokens)

bank_data2 = pd.DataFrame({'text':X_train, 'tokenizer_text': X_train.apply(tokenize_text)})
bank_data2



bank_data2.head()

from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
bank_data['Score'] = bank_data['Score'].apply(lambda x: 1 if x == 'Positive' else 0)

In [5]:
import pandas as pd
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [6]:
X_train, X_test, y_train, y_test = train_test_split(bank_data['Text'], bank_data['Score'], test_size=0.5, random_state=35)

vectorizer = TfidfVectorizer()
x_train_vectorized = vectorizer.fit_transform(X_train.apply(tokenize_text).apply(lambda x: ' '.join(x)))


support_vector_machine_model = SVC()
support_vector_machine_model.fit(x_train_vectorized, y_train)


bank_new_sentences = [
    "Хорошие консультанты и  выгодные предложения мы берем ипотеку.",
    "Банк плохой консультанты грубили мне я отказываюсь от ипотеки."
]

bank_new_sentences_vectorized = vectorizer.transform(pd.Series(bank_new_sentences).apply(tokenize_text).apply(lambda x: ' '.join(x)))
svm_predictions = support_vector_machine_model.predict(bank_new_sentences_vectorized)

for sentence, prediction in zip(bank_new_sentences, svm_predictions):
    if prediction == 1:
        print(f"Sentence: {sentence}\nКлассификация: Positive")
    else:
        print(f"Sentence: {sentence}\nКлассификация: Negative")

x_test_vectorized = vectorizer.transform(X_test)
svm_predictions = support_vector_machine_model.predict(x_test_vectorized)
svm_model_accuracy = accuracy_score(y_test, svm_predictions)
print(f"Accuracy: {svm_model_accuracy}")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Предложение: Хорошие консультанты и  выгодные предложения мы берем ипотеку.
Классификация: Positive

Предложение: Банк плохой консультанты грубили мне я отказываюсь от ипотеки.
Классификация: Negative

Точность модели: 0.9469924274896414


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a fut

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(bank_data['Text'], bank_data['Score'], test_size=0.5, random_state=35)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_tokenized)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=X_train_padded.shape[1])

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 100, input_length=X_train_padded.shape[1]),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_padded, y_train_encoded, epochs=5, batch_size=32)

bank_new_sentences = [
    "Хорошие консультанты и  выгодные предложения мы берем ипотеку.",
    "Банк плохой консультанты грубили мне я отказываюсь от ипотеки."
]

bank_new_sentences_vectorized = vectorizer.transform(pd.Series(bank_new_sentences).apply(tokenize_text).apply(lambda x: ' '.join(x)))
svm_predictions = support_vector_machine_model.predict(bank_new_sentences_vectorized)

for sentence, prediction in zip(bank_new_sentences, svm_predictions):
    if prediction == 1:
        print(f"Sentence: {sentence}\nКлассификация: Positive")
    else:
        print(f"Sentence: {sentence}\nКлассификация: Negative")
_, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print('Accuracy: %.2f' % (accuracy * 100))

ModuleNotFoundError: No module named 'tensorflow'